### Indexes and index sets

* $n \in N$: Set of nodes
* $i \in G$: Set of generators. Each generator is associated with a node $n$.
* $b \in B$: Set of branches
* $b \in B_n^{in}$: Set of branches coming into node $n$
* $b \in B_n^{out}$: Set of branches going out of node $n$
* $t \in T$: Set of time periods
* $i \in G_n$: Set of generators at node $n$

### Parameters

* $P_{i}^{\min}$: Minimum power output of generator $i$ (MW)
* $P_{i}^{\max}$: Maximum power output of generator $i$ (MW)
* ${VOLL}$: Value of lost load (cost of load shedding) ($/MWh)
* $MC_{i}$: Marginal cost of generator $i$ (\$/MWh)
* $CO2_{i}$ Cost of CO2 emissions of generator $i$ (\$/MWh)
* $E_{i}$: CO2 emissions of generator $i$ (ton/MWh) 
* $E_{limit}$: CO2 emissions limit (ton)
* $D_{n,t}$: Demand at node $n$ and time $t$ (MW)
* $l_b$: loss factor of branch $b$ (given, but in reality some function of distance, transmsission line type, etc.)
* $P_{b}^{\max}$: Maximum power flow on branch $b$ (MW)

### Decision variables
* $g_{i,t}$: Power generation dispatch of generator $i$ at time $t$ (MW)
* $f_{b,t}$: Power flow on branch $b$ at time $t$ (MW)
* $s_{n,t}$: Load shedding at node $n$ at time $t$ (MW)
* $c_{n,t}$: Power curtailment at node $n$ at time $t$ (MW)

### Optimization Model

### Objective function, minimize cost of generation

**Minimize:**
$$ \sum_{i \in G} \sum_{t \in T} (MC_i + CO2_i) P_{i,t} + \sum_{n \in N} \sum_{t \in T} s_{n,t} VOLL $$

1. **Power balance: production + inflow - curtailment = outflow + demand - shedding**

*A.K.A. Market clearing or energy balance*

$$ \sum_i (g_{i,t} - c_{i,t}) + \sum_{b \in B_n^{in}} f_{b,t}(1-l_b) - \sum_{b \in B_n^{out}} f_{b,t} + s_{n,t} = D_{n,t} \quad \forall n \in N, \forall t \in T $$

2. **We can't shed more load than the demand**

$$ s_{n,t} \leq D_{n,t} \quad \forall n \in N, \forall t \in T $$

3. **Generators' power output limits**

$$ P_{i}^{\min} \leq g_{i,t} \leq P_{i}^{\max} \quad \forall i \in G, \forall t \in T $$

4. **Branch power flow limits**

$$ -P_{b}^{\max} \leq f_{b,t} \leq P_{b}^{\max} \quad \forall b \in B, \forall t \in T $$

5. **Emissions restrictions**

$$ \sum_i \sum_t E_i g_{i,t} \leq E_{limit} $$

6. **Variable definitions**

$$ g_{i,t} \geq 0, \quad f_{b,t} \geq 0, \quad s_{n,t} \geq 0, \quad c_{n,t} \geq 0 \quad \forall i \in G, \forall b \in B, \forall n \in N, \forall t \in T $$

# Basic model implementation

In [23]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import matplotlib.pyplot as plt
import os
import numpy as np
np.random.seed(0)

In [24]:
DATA_FOLDER = os.path.join(os.path.dirname(os.getcwd()), 'data')

In [29]:
branches = pd.read_csv(os.path.join(DATA_FOLDER, 'branches.csv'))
nodes = pd.read_csv(os.path.join(DATA_FOLDER, 'nodes.csv'))
generators = pd.read_csv(os.path.join(DATA_FOLDER, 'generators.csv'))
hourly_demand = pd.read_csv(os.path.join(DATA_FOLDER, 'hourly_demand.csv'))
VOLL = 1000  # Example value for Value of Lost Load (set appropriately)


In [38]:
# Initialize model
model = gp.Model("power_flow_optimization")

# Sets
N = nodes['node_id'].tolist()  # Set of nodes
G = generators['generator_id'].tolist()  # Set of generators
B = branches['branch_id'].tolist()  # Set of branches
T = hourly_demand['hour'].unique().tolist()  # Set of time periods

# Parameters
P_min = {row['generator_id']: row['pmin'] for _, row in generators.iterrows()}  # Minimum generation
P_max = {row['generator_id']: row['pmax'] for _, row in generators.iterrows()}  # Maximum generation
MC = {row['generator_id']: row['MC'] for _, row in generators.iterrows()}  # Marginal cost
CO2 = {row['generator_id']: row['CO2'] for _, row in generators.iterrows()}  # CO2 cost
D = {(row['node_id'], row['hour']): row['demand'] for _, row in hourly_demand.iterrows()}  # Demand at node n, time t
E = {row['generator_id']: row['CO2'] for _, row in generators.iterrows()}  # Emissions factor
E_limit = 1000  # Example CO2 emissions limit
P_b_max = {row['branch_id']: row['capacity'] for _, row in branches.iterrows()}  # Branch power flow limits
loss_factor = {row['branch_id']: row['loss_factor'] for _, row in branches.iterrows()}  # Loss factor for each branch

# Decision Variables
g = model.addVars(G, T, lb=0, ub=GRB.INFINITY, name="generation")  # Generation dispatch
f = model.addVars(B, T, lb=-GRB.INFINITY, ub=GRB.INFINITY, name="flow")  # Power flow
s = model.addVars(N, T, lb=0, name="shedding")  # Load shedding
c = model.addVars(N, T, lb=0, name="curtailment")  # Power curtailment

# Objective: Minimize total generation cost (MC + CO2 cost + shed cost)
obj = gp.quicksum((MC[i] + CO2[i]) * g[i, t] for i in G for t in T) + gp.quicksum(VOLL * s[n, t] for n in N for t in T)
model.setObjective(obj, GRB.MINIMIZE)

# Constraint 1: Power balance at each node and time
for n in N:
    for t in T:
        inflow = gp.quicksum(f[b, t] * (1 - loss_factor[b]) for b in branches[branches['node_to'] == n]['branch_id'])
        outflow = gp.quicksum(f[b, t] for b in branches[branches['node_from'] == n]['branch_id'])
        generation = gp.quicksum(g[i, t] - c[i, t] for i in generators[generators['node_id'] == n]['generator_id'])
        demand = D[n, t]
        model.addConstr(generation + inflow - outflow + s[n, t] == demand, f"power_balance_{n}_{t}")

# Constraint 2: Load shedding can't exceed demand
for n in N:
    for t in T:
        model.addConstr(s[n, t] <= D[n, t], f"shedding_limit_{n}_{t}")

# Constraint 3: Generator output limits
for i in G:
    for t in T:
        model.addConstr(g[i, t] >= P_min[i], f"gen_min_{i}_{t}")
        model.addConstr(g[i, t] <= P_max[i], f"gen_max_{i}_{t}")

# Constraint 4: Branch power flow limits
for b in B:
    for t in T:
        model.addConstr(f[b, t] <= P_b_max[b], f"branch_max_{b}_{t}")
        model.addConstr(f[b, t] >= -P_b_max[b], f"branch_min_{b}_{t}")

# Constraint 5: CO2 emissions limit
model.addConstr(gp.quicksum(E[i] * g[i, t] for i in G for t in T) <= E_limit, "emissions_limit")

# Optimize the model
model.optimize()

# Extract the results
if model.status == GRB.OPTIMAL:
    generation = model.getAttr('x', g)
    flow = model.getAttr('x', f)
    shedding = model.getAttr('x', s)
    curtailment = model.getAttr('x', c)
    
    # Display results as needed


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 3073 rows, 2016 columns and 5376 nonzeros
Model fingerprint: 0xf2981b89
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [5e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-01, 1e+03]
Presolve removed 2592 rows and 506 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Infeasible model


In [36]:
MC

{'Solar in A2': 5.192127132363674,
 'Oil in A2': 73.22701068302649,
 'Oil in A1': 54.49024601551049,
 'Coal in A2': 23.418191702720904,
 'Solar in A1': 7.331553864281531}